In [3]:
import os
os.environ["FLASK_ENV"]='development'

import globus_automate_client
from mdf_connect_server.automate.globus_automate_flow import GlobusAutomateFlow
import mdf_toolbox
import json
import time

In [5]:
from mdf_connect_server.automate.globus_automate_flow import GlobusAutomateFlow  # NOQA

native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"
client = globus_automate_client.create_flows_client(native_app_id)
mdf_flow = GlobusAutomateFlow.from_existing_flow(client, "mdf_flow_info.json")

print(mdf_flow)

Globus Automate Flow: id=cc7d1112-104c-419b-a159-81c9887d420d, scope=https://auth.globus.org/scopes/cc7d1112-104c-419b-a159-81c9887d420d/flow_cc7d1112_104c_419b_a159_81c9887d420d_user


In [7]:
# Generate input credentials
# The feedstock auth header is shorter-lived, so it's not set in the Flow defintition.
# However, the header must be from an identity with access to the Xtract output location (currently Petrel)
petrel_auth = mdf_toolbox.login(services=["petrel"], make_clients=False)["petrel"]
feedstock_auth_header = {}
petrel_auth.set_authorization_header(feedstock_auth_header)
feedstock_auth_header = feedstock_auth_header["Authorization"]

# The RunAs token is used to Transfer data from the user as the user's identity.
# It will be given by the MDF user (by logging in with MDF).
# NOTE: Currently, RunAs is not used on the Transfer.
run_as_auth = mdf_flow.get_runas_auth()
run_as_token = run_as_auth.refresh_token

In [8]:
# Get test input for Flow
with open("mdf_flow_input.json") as f:
    flow_input = json.load(f)

# Add credentials to input
flow_input["_private_feedstock_auth_header"] = feedstock_auth_header
flow_input["_tokens"] = {"MDFUser": run_as_token}

# Create unique source_id for submission (source_id must be unique except for updates)
source_id = "mdf_flow_test_" + str(int(time.time())) + "_v1.1"
flow_input["source_id"] = source_id
source_id

'mdf_flow_test_1607114852_v1.1'

In [9]:
flow = mdf_flow.run_flow(flow_input)

In [10]:
print(json.dumps(flow.get_status(), indent=4, sort_keys=True))

{
    "action_id": "8702f9bf-bd69-46fe-a7e4-650eea98f409",
    "completion_time": "2020-12-04T20:49:37.987000+00:00",
    "created_by": "urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b",
    "details": {
        "output": {
            "UndoUserPermissionResult": {},
            "UserPermissionResult": {},
            "UserTransferResult": {
                "action_id": "09a9ae3f-0615-4afd-8904-110c0ecf740e",
                "completion_time": "2020-12-04T20:49:25.394000+00:00",
                "created_by": "urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b",
                "details": {
                    "output": {
                        "TransferResult": {
                            "action_id": "1MOrQctV78G74",
                            "completion_time": "2020-12-04T20:49:24.188486+00:00",
                            "creator_id": "urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b",
                            "details": {
             

In [11]:
flow.get_error_msgs()

[]

In [8]:
flow.action_id

'a141115e-f938-4111-8b6f-769af74c9f94'

In [9]:
automate

In [10]:
print(automate)

Globus Automate Flow: id=33af393f-4c5d-4031-ba2d-0c898de7c4fd, scope=https://auth.globus.org/scopes/33af393f-4c5d-4031-ba2d-0c898de7c4fd/flow_33af393f_4c5d_4031_ba2d_0c898de7c4fd_user
